<a href="https://colab.research.google.com/github/LatiefDataVisionary/data-mining-college-task/blob/main/Tgs_Kel_Data_Mining_Week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# Load the data file
file_path = 'SampleSuperstore.csv'
data = pd.read_csv(file_path)
data

,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,25.2480,3,0.20,4.1028
9990,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,91.9600,2,0.00,15.6332
9991,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,258.5760,2,0.20,19.3932
9992,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,29.6000,4,0.00,13.3200


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ship Mode     9994 non-null   object 
 1   Segment       9994 non-null   object 
 2   Country       9994 non-null   object 
 3   City          9994 non-null   object 
 4   State         9994 non-null   object 
 5   Postal Code   9994 non-null   int64  
 6   Region        9994 non-null   object 
 7   Category      9994 non-null   object 
 8   Sub-Category  9994 non-null   object 
 9   Sales         9994 non-null   float64
 10  Quantity      9994 non-null   int64  
 11  Discount      9994 non-null   float64
 12  Profit        9994 non-null   float64
dtypes: float64(3), int64(2), object(8)
memory usage: 1015.1+ KB


In [ ]:
# Create an SQLite database
engine = create_engine('sqlite:///superstore_data_warehouse.db')

In [ ]:
# Step 1: Create dimensional tables
dim_customer = data[['Segment', 'Country', 'City', 'State', 'Postal Code', 'Region']].drop_duplicates()
dim_customer.reset_index(drop=True, inplace=True)
dim_customer['CustomerID'] = dim_customer.index + 1

In [ ]:
dim_product = data[['Category', 'Sub-Category']].drop_duplicates()
dim_product.reset_index(drop=True, inplace=True)
dim_product['ProductID'] = dim_product.index + 1

dim_shipping = data[['Ship Mode']].drop_duplicates()
dim_shipping.reset_index(drop=True, inplace=True)
dim_shipping['ShippingID'] = dim_shipping.index + 1

In [ ]:
# Step 2: Create fact table by merging IDs from dimension tables
fact_sales = data.merge(dim_customer, how='left', on=['Segment', 'Country', 'City', 'State', 'Postal Code', 'Region'])
fact_sales = fact_sales.merge(dim_product, how='left', on=['Category', 'Sub-Category'])
fact_sales = fact_sales.merge(dim_shipping, how='left', on=['Ship Mode'])

fact_sales = fact_sales[['CustomerID', 'ProductID', 'ShippingID', 'Sales', 'Quantity', 'Discount', 'Profit']]

In [ ]:
# Step 3: Save tables to SQLite database
dim_customer.to_sql('dim_customer', engine, index=False, if_exists='replace')
dim_product.to_sql('dim_product', engine, index=False, if_exists='replace')
dim_shipping.to_sql('dim_shipping', engine, index=False, if_exists='replace')
fact_sales.to_sql('fact_sales', engine, index=False, if_exists='replace')

9994

In [ ]:
print("Data warehouse created successfully with the following tables:")
print("- dim_customer")
print("- dim_product")
print("- dim_shipping")
print("- fact_sales")

Data warehouse created successfully with the following tables:
- dim_customer
- dim_product
- dim_shipping
- fact_sales


In [ ]:
# Validation Queries
def validate_tables():
    conn = sqlite3.connect('superstore_data_warehouse.db')
    print("\nSample from dim_customer:")
    print(pd.read_sql_query("SELECT * FROM dim_customer LIMIT 5;", conn))

    print("\nSample from dim_product:")
    print(pd.read_sql_query("SELECT * FROM dim_product LIMIT 5;", conn))

    print("\nSample from dim_shipping:")
    print(pd.read_sql_query("SELECT * FROM dim_shipping LIMIT 5;", conn))

    print("\nSample from fact_sales:")
    print(pd.read_sql_query("SELECT * FROM fact_sales LIMIT 5;", conn))
    conn.close()

# Run validation
validate_tables()


Sample from dim_customer:
     Segment        Country             City           State  Postal Code  \
0   Consumer  United States        Henderson        Kentucky        42420   
1  Corporate  United States      Los Angeles      California        90036   
2   Consumer  United States  Fort Lauderdale         Florida        33311   
3   Consumer  United States      Los Angeles      California        90032   
4   Consumer  United States          Concord  North Carolina        28027   

  Region  CustomerID  
0  South           1  
1   West           2  
2  South           3  
3   West           4  
4  South           5  

Sample from dim_product:
          Category Sub-Category  ProductID
0        Furniture    Bookcases          1
1        Furniture       Chairs          2
2  Office Supplies       Labels          3
3        Furniture       Tables          4
4  Office Supplies      Storage          5

Sample from dim_shipping:
        Ship Mode  ShippingID
0    Second Class           1
1

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('superstore_data_warehouse.db')
cursor = conn.cursor()

In [ ]:
# 1. Total Sales by Region (Rool=up)
query_total_sales_region = """
SELECT c.Region, SUM(s.Sales) AS TotalSales
FROM fact_sales s
JOIN dim_customer c ON s.CustomerID = c.CustomerID
GROUP BY c.Region
ORDER BY TotalSales DESC;
"""
df_total_sales_region = pd.read_sql_query(query_total_sales_region, conn)
print("Total Sales by Region:")
print(df_total_sales_region)

Total Sales by Region:
    Region   TotalSales
0     West  725457.8245
1     East  678781.2400
2  Central  501239.8908
3    South  391721.9050


In [ ]:
# 2. Average Profit by Product Category
query_avg_profit_category = """
SELECT p.Category, AVG(s.Profit) AS AverageProfit
FROM fact_sales s
JOIN dim_product p ON s.ProductID = p.ProductID
GROUP BY p.Category
ORDER BY AverageProfit DESC;
"""
df_avg_profit_category = pd.read_sql_query(query_avg_profit_category, conn)
print("\nAverage Profit by Product Category:")
print(df_avg_profit_category)


Average Profit by Product Category:
          Category  AverageProfit
0       Technology      78.752002
1  Office Supplies      20.327050
2        Furniture       8.699327


In [ ]:
# 3. Total Quantity Sold per Sub-Category
query_total_quantity_subcategory = """
SELECT p."Sub-Category", SUM(s.Quantity) AS TotalQuantitySold
FROM fact_sales s
JOIN dim_product p ON s.ProductID = p.ProductID
GROUP BY p."Sub-Category"
ORDER BY TotalQuantitySold DESC;
"""
df_total_quantity_subcategory = pd.read_sql_query(query_total_quantity_subcategory, conn)
print("\nTotal Quantity Sold per Sub-Category:")
print(df_total_quantity_subcategory)


Total Quantity Sold per Sub-Category:
   Sub-Category  TotalQuantitySold
0       Binders               5974
1         Paper               5178
2   Furnishings               3563
3        Phones               3289
4       Storage               3158
5           Art               3000
6   Accessories               2976
7        Chairs               2356
8    Appliances               1729
9        Labels               1400
10       Tables               1241
11    Fasteners                914
12    Envelopes                906
13    Bookcases                868
14     Supplies                647
15     Machines                440
16      Copiers                234


In [ ]:
# 4. Total Sales by Ship Mode
query_total_sales_shipmode = """
SELECT sh."Ship Mode", SUM(s.Sales) AS TotalSales
FROM fact_sales s
JOIN dim_shipping sh ON s.ShippingID = sh.ShippingID
GROUP BY sh."Ship Mode"
ORDER BY TotalSales DESC;
"""
df_total_sales_shipmode = pd.read_sql_query(query_total_sales_shipmode, conn)
print("\nTotal Sales by Ship Mode:")
print(df_total_sales_shipmode)


Total Sales by Ship Mode:
        Ship Mode    TotalSales
0  Standard Class  1.358216e+06
1    Second Class  4.591936e+05
2     First Class  3.514284e+05
3        Same Day  1.283631e+05


In [ ]:
# Close the connection
conn.close()

## **OLAP**

In [ ]:
# Koneksi ke database
conn = sqlite3.connect('superstore_data_warehouse.db')

In [ ]:
conn = sqlite3.connect('superstore_data_warehouse.db')
cursor = conn.cursor()

In [ ]:
query_rollup = """
SELECT Region, SUM(Sales) AS TotalSales
FROM fact_sales
JOIN dim_customer ON fact_sales.CustomerID = dim_customer.CustomerID
GROUP BY Region;
"""

df_rollup = pd.read_sql_query(query_rollup, conn)
df_rollup

,Region,TotalSales
0,Central,501239.8908
1,East,678781.2400
2,South,391721.9050
3,West,725457.8245


In [ ]:
# Step 1: Create a Date Dimension
dim_date = data[['Order Date']].drop_duplicates()
dim_date.reset_index(drop=True, inplace=True)
dim_date['DateID'] = dim_date.index + 1
dim_date.rename(columns={'Order Date': 'OrderDate'}, inplace=True)  # Rename column for consistency

# Step 2: Add DateID to fact_sales
fact_sales = data.merge(dim_customer, how='left', on=['Segment', 'Country', 'City', 'State', 'Postal Code', 'Region'])
fact_sales = fact_sales.merge(dim_product, how='left', on=['Category', 'Sub-Category'])
fact_sales = fact_sales.merge(dim_shipping, how='left', on=['Ship Mode'])
fact_sales = fact_sales.merge(dim_date, how='left', on=['OrderDate']) # Merge with date dimension
fact_sales = fact_sales[['CustomerID', 'ProductID', 'ShippingID', 'DateID', 'Sales', 'Quantity', 'Discount', 'Profit']]

# ... (rest of the data loading code)

# Step 3: Save the date dimension table
dim_date.to_sql('dim_date', engine, index=False, if_exists='replace')

# ... (rest of the code)

KeyError: "None of [Index(['Order Date'], dtype='object')] are in the [columns]"

In [ ]:
# Step 4: Modify the query_drilldown:
query_drilldown = """
SELECT strftime('%Y-%m', d.OrderDate) AS Month, SUM(s.Sales) AS TotalSales
FROM fact_sales s
JOIN dim_customer c ON s.CustomerID = c.CustomerID
JOIN dim_date d ON s.DateID = d.DateID   # Join with date dimension
WHERE c.Region = 'East'
GROUP BY strftime('%Y-%m', d.OrderDate);
"""
df_drilldown = pd.read_sql_query(query_drilldown, conn)
print(df_drilldown)

DatabaseError: Execution failed on sql '
SELECT strftime('%Y-%m', OrderDate) AS Month, SUM(Sales) AS TotalSales
FROM fact_sales 
JOIN dim_customer ON fact_sales.CustomerID = dim_customer.CustomerID
WHERE Region = 'East'
GROUP BY strftime('%Y-%m', OrderDate);
': no such column: OrderDate

In [ ]:
query_slice = """
SELECT Category, SUM(Sales) AS TotalSales, SUM(Profit) AS TotalProfit
FROM fact_sales
JOIN dim_product ON fact_sales.ProductID = dim_product.ProductID
WHERE Category = 'Furniture'
GROUP BY Category;
"""

df_slice = pd.read_sql_query(query_slice, conn)
print(df_slice)

ProgrammingError: Cannot operate on a closed database.

In [ ]:
query_dice = """
SELECT Region, Category, SUM(Sales) AS TotalSales, SUM(Profit) AS TotalProfit
FROM fact_sales
JOIN dim_customer ON fact_sales.CustomerID = dim_customer.CustomerID
JOIN dim_product ON fact_sales.ProductID = dim_product.ProductID
WHERE Region = 'West' AND Category = 'Technology'
GROUP BY Region, Category;
"""

df_dice = pd.read_sql_query(query_dice, conn)
print(df_dice)

ProgrammingError: Cannot operate on a closed database.